<a href="https://colab.research.google.com/github/fernandojunior/deteccao-fraude/blob/master/src/Fraud_Detection_Xente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spark load lib

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('xente').getOrCreate()

In [0]:
!pip install wget==3.2
spark

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=d4cf6f3e595baff1a4373d08604badbb397990eb022f47e42259af3959a28ba0
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Load Training Data

In [0]:
import pandas as pd
training_data = pd.read_csv("https://drive.google.com/uc?export=download&id=1NrtVkKv8n_g27w5elq9HWZA1i8aFBW0G")
training_data.head()
df = spark.createDataFrame(training_data)

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


## Dicionário dos Dados

*   **TransactionId:** Unique transaction identifier on platform.
*   **BatchId:** Unique number assigned to a batch of transactions for processing.
*   **AccountId:** Unique number identifying the customer on platform.
*   **SubscriptionId:** Unique number identifying the customer subscription.
*   **CustomerId:** Unique identifier attached to Account.
*   **CurrencyCode:** Country currency.
*   **CountryCode:** Numerical geographical code of country.
*   **ProviderId:** Source provider of Item bought.
*   **ProductId:** Item name being bought.
*   **ProductCategory:** ProductIds are organized into these broader product categories.
*   **ChannelId:** Identifies if customer used web,Android, IOS, pay later or checkout.
*   **Amount:** Value of the transaction. Positive for debits from customer account and negative for credit into customer account.
*   **Value:** Absolute value of the amount.
*   **TransactionStartTime:** Transaction start time.
*   **PricingStrategy:** Category of Xente's pricing structure for merchants.
*   **FraudResult:** Fraud status of transaction: 1) Yes; or 0) No.

In [0]:
df.printSchema()

root
 |-- TransactionId: string (nullable = true)
 |-- BatchId: string (nullable = true)
 |-- AccountId: string (nullable = true)
 |-- SubscriptionId: string (nullable = true)
 |-- CustomerId: string (nullable = true)
 |-- CurrencyCode: string (nullable = true)
 |-- CountryCode: long (nullable = true)
 |-- ProviderId: string (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- ProductCategory: string (nullable = true)
 |-- ChannelId: string (nullable = true)
 |-- Amount: double (nullable = true)
 |-- Value: long (nullable = true)
 |-- TransactionStartTime: string (nullable = true)
 |-- PricingStrategy: long (nullable = true)
 |-- FraudResult: long (nullable = true)



## Análise dos Dados Faltantes

In [0]:
print(df.count())
df.na.drop(how='any').count()

95662


95662

**Não** há dados faltantes no dataset de **treinamento** : https://zindi.africa/competitions/xente-fraud-detection-challenge/data


### Há linhas duplicadas?



In [0]:
df = df.distinct()
print(df.count())

95662


Como pode ser observado, o número de linhas são iguais, portanto, todas as linhas são distintas.

## Estatísticas Descritivas

In [0]:
training_data.describe()

,CountryCode,Amount,Value,PricingStrategy,FraudResult
count,95662.0,9.566200e+04,9.566200e+04,95662.000000,95662.000000
mean,256.0,6.717846e+03,9.900584e+03,2.255974,0.002018
std,0.0,1.233068e+05,1.231221e+05,0.732924,0.044872
min,256.0,-1.000000e+06,2.000000e+00,0.000000,0.000000
25%,256.0,-5.000000e+01,2.750000e+02,2.000000,0.000000
50%,256.0,1.000000e+03,1.000000e+03,2.000000,0.000000
75%,256.0,2.800000e+03,5.000000e+03,2.000000,0.000000
max,256.0,9.880000e+06,9.880000e+06,4.000000,1.000000


In [0]:
df.groupBy('FraudResult').agg(df.count('*')).show()

TypeError: ignored

### Associacao Fraude-Transacao

In [0]:
group_fraud = df.groupBy('FraudResult')

In [0]:
from pyspark.sql.functions import countDistinct
df.select(countDistinct('FraudResult').alias('I/O')).show()

+---+
|I/O|
+---+
|  2|
+---+



In [0]:
print('Quantity of frauds: ' + str(df.filter(df['FraudResult'] == 1).count()) )

example_fraud = df.filter(df['FraudResult'] == 1).collect()
example_fraud[0]

Quantity of frauds: 193


Row(TransactionId='TransactionId_60873', BatchId='BatchId_93685', AccountId='AccountId_3832', SubscriptionId='SubscriptionId_4161', CustomerId='CustomerId_4275', CurrencyCode='UGX', CountryCode=256, ProviderId='ProviderId_3', ProductId='ProductId_15', ProductCategory='financial_services', ChannelId='ChannelId_3', Amount=700000.0, Value=700000, TransactionStartTime='2018-11-15T08:04:51Z', PricingStrategy=0, FraudResult=1)

In [0]:
#TransactionId = TransactionId_76871
#TransactionId = 76871
example_fraud.split(df['TransactionId'], '_')[1]

## Análise de Normalidade

## Análise e Tratamento de Outliers

## Análise de Variância


## Análise de Correlação

## Análise e Tratamento dos dados desbalanceados

# Preparação dos Dados

## Conversão dos dados Categóricos

## Normalização dos Dados

## Extração de características

# Modelagem

# Avaliação


In [0]:
import pandas as pd
test_data = pd.read_csv("https://drive.google.com/uc?export=download&id=16cRQIFW6n2th2YOK7DEsp9dQgihHDuHa")
test_data.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy
0,TransactionId_50600,BatchId_35028,AccountId_2441,SubscriptionId_4426,CustomerId_2857,UGX,256,ProviderId_5,ProductId_3,airtime,ChannelId_3,1000.0,1000,2019-02-13T10:01:40Z,4
1,TransactionId_95109,BatchId_45139,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,256,ProviderId_5,ProductId_15,financial_services,ChannelId_3,2000.0,2000,2019-02-13T10:02:12Z,2
2,TransactionId_47357,BatchId_74887,AccountId_4841,SubscriptionId_3829,CustomerId_2857,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-50.0,50,2019-02-13T10:02:30Z,2
3,TransactionId_28185,BatchId_11025,AccountId_2685,SubscriptionId_4626,CustomerId_3105,UGX,256,ProviderId_5,ProductId_10,airtime,ChannelId_3,3000.0,3000,2019-02-13T10:02:38Z,4
4,TransactionId_22140,BatchId_29804,AccountId_4841,SubscriptionId_3829,CustomerId_3105,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-60.0,60,2019-02-13T10:02:58Z,2


### Análise de dados 

In [0]:
df_test = spark.createDataFrame(test_data)
print(df_test.count())
df_test.na.drop(how='any').count()

45019


45019